# 人在回路

在使用 Bridgic 构建工作流或代理时，开发人员可以无缝地将人在回路的交互集成到执行流程中。在任何时候，系统都可以暂停其自动化过程以请求人类输入——例如批准、验证或额外指令——并等待响应。一旦提供了人类反馈，工作流或代理将从中断点恢复执行，并根据新的输入调整其行为。Bridgic 确保整个过程，包括暂停和恢复状态，可以可靠地序列化和反序列化，以便进行持久化和恢复。

## 交互场景

让我们通过几个简单的例子来理解这个过程。在此之前，让我们设置运行环境。

运行以下 `pip` 命令以确保 ['openai' 集成](../../../../reference/bridgic-llms-openai/bridgic/llms/openai/) 已安装。

```shell
pip install -U bridgic
pip install -U bridgic-llms-openai
```

In [ ]:
import os

# Get the API base, API key and model name.
_api_key = os.environ.get("OPENAI_API_KEY")
_api_base = os.environ.get("OPENAI_API_BASE")
_model_name = os.environ.get("OPENAI_MODEL_NAME")

from pydantic import BaseModel, Field
from bridgic.core.automa import GraphAutoma, worker, Snapshot
from bridgic.core.automa.args import From
from bridgic.core.automa.interaction import Event, Feedback, FeedbackSender, InteractionFeedback, InteractionException
from bridgic.core.model.types import Message, Role
from bridgic.core.model.protocols import PydanticModel
from bridgic.llms.openai import OpenAILlm

### 编程助手

在开发编程助手的过程中，可以设计它自动执行和验证生成的代码。然而，由于程序执行会消耗系统资源，用户必须决定是否允许助手运行代码。

让我们通过 Bridgic 来实现这一点。可以在 GitHub 上下载 [源代码](https://github.com/bitsky-tech/bridgic-examples/blob/main/human_in_the_loop/code_assistant.py)。步骤如下：

1. 根据用户需求生成代码。
2. 询问用户是否允许执行生成的代码。
3. 输出结果。

In [25]:
# Set the LLM
llm = OpenAILlm(api_base=_api_base, api_key=_api_key, timeout=10)

class CodeBlock(BaseModel):
    code: str = Field(description="The code to be executed.")

class CodeAssistant(GraphAutoma):
    @worker(is_start=True)
    async def generate_code(self, user_requirement: str):
        response = await llm.astructured_output(
            model=_model_name,
            messages=[
                Message.from_text(text=f"You are a programming assistant. Please generate code according to the user's requirements.", role=Role.SYSTEM),
                Message.from_text(text=user_requirement, role=Role.USER),
            ],
            constraint=PydanticModel(model=CodeBlock)
        )
        return response.code

    @worker(dependencies=["generate_code"])
    async def ask_to_run_code(self, code: str):
        event = Event(event_type="can_run_code", data=code)
        feedback = await self.request_feedback_async(event)
        return feedback.data
        
    @worker(dependencies=["ask_to_run_code"])
    async def output_result(self, feedback: str, code: str = From("generate_code")):
        code = code.strip("```python").strip("```")
        if feedback == "yes":
            print(f"- - - - - - Result - - - - - -")
            exec(code)
            print(f"- - - - - - End - - - - - -")
        else:
            print(f"This code was rejected for execution. In response to the requirements, I have generated the following code:\n```python\n{code}\n```")

在 `CodeAssistant` 的 `ask_to_run_code()` 方法中，我们使用 [`request_feedback_async()`](../../../../reference/bridgic-core/bridgic/core/automa/#bridgic.core.automa.Automa.request_feedback_async) 向人类用户发送事件，并期望收到反馈。要处理此事件，需要将相应的逻辑注册到 automa，如下所示：

In [26]:
# Handle can_run_code event
def can_run_code_handler(event: Event, feedback_sender: FeedbackSender):
    print(f"Can I run this code now to verify if it's correct?")
    print(f"```python\n{event.data}\n```")
    res = input("Please input your answer (yes/no): ")
    if res in ["yes", "no"]:
        feedback_sender.send(Feedback(data=res))
    else:
        print("Invalid input. Please input yes or no.")
        feedback_sender.send(Feedback(data="no"))

# register can_run_code event handler to `CodeAssistant` automa
code_assistant = CodeAssistant()
code_assistant.register_event_handler("can_run_code", can_run_code_handler)

现在让我们开始使用它！

In [27]:
await code_assistant.arun(user_requirement="Please write a function to print 'Hello, World!' and run it.")

Can I run this code now to verify if it's correct?
```python
def greet():
    print('Hello, World!')

greet()
```
- - - - - - Result - - - - - -
Hello, World!
- - - - - - End - - - - - -


在上述示例中，Bridgic 将发送给人类用户的消息包装在一个 [`Event`](../../../../reference/bridgic-core/bridgic/core/automa/interaction/#bridgic.core.automa.interaction.Event) 中，并将从用户接收到的消息包装在一个 [`FeedBack`](../../../../reference/bridgic-core/bridgic/core/automa/interaction/#bridgic.core.automa.interaction.Feedback) 中。

- `Event` 包含三个字段：
    - `event_type`：一个字符串。事件类型用于识别注册的事件处理程序。
    - `timestamp`：一个 Python datetime 对象。事件的时间戳。默认值为 `datetime.now()`。
    - `data`：附加到事件的数据。
- `FeedBack` 包含一个字段：
    - `data`：附加到反馈的数据。

`request_feedback_async()` 向用户发送事件并请求反馈。此方法调用将阻塞调用者，直到收到反馈。然而，得益于 Python 的异步事件循环机制，运行在同一主线程上的其他 automas 不会被阻塞。

注册的事件处理程序必须定义为 [`EventHandlerType`](../../../../reference/bridgic-core/bridgic/core/automa/interaction/#bridgic.core.automa.interaction.EventHandlerType) 类型。这里应该是一个接受 [`Event`](../../../../reference/bridgic-core/bridgic/core/automa/interaction/#bridgic.core.automa.interaction.Event) 和 [`FeedbackSender`](../../../../reference/bridgic-core/bridgic/core/automa/interaction/#bridgic.core.automa.interaction.FeedbackSender) 作为参数的函数。

### 计数通知器

有时，可能需要发布一个事件而不期望任何反馈，例如消息通知或进度更新。在这种情况下，我们调用 [`post_event()`](../../../../reference/bridgic-core/bridgic/core/automa/#bridgic.core.automa.Automa.post_event) 方法，并注册一个类型为 [`EventHandlerType`](../../../../reference/bridgic-core/bridgic/core/automa/interaction/#bridgic.core.automa.interaction.EventHandlerType) 的事件处理程序来处理该事件。这里的事件处理程序应该是一个只接受 `Event` 作为参数的函数。

例如，计数通知器的实现是从 1 计数到 `user_input` 参数指定的数字。用户还可以指定哪个数字 (`notify_int`) 应该触发提醒。

In [ ]:
class MessageNotifier(GraphAutoma):
    @worker(is_start=True)
    async def notify(self, user_input: int, notify_int: int):
        print(f"Loop from 1 to {user_input}")
        for i in range(1, user_input + 1):
            if i == notify_int:
                event = Event(event_type="message_notification", data=f"Loop {i} times")
                self.post_event(event)

def message_notification_handler(event: Event):
    print(f'!! Now count to {event.data}. !!')

message_notifier = MessageNotifier()
message_notifier.register_event_handler("message_notification", message_notification_handler)
await message_notifier.arun(user_input=10, notify_int=5)
        

Loop from 1 to 10
!! Now count to Loop 5 times. !!


### 报销工作流

在某些情况下，触发事件后可能需要长时间等待反馈。然而，保持系统处于等待状态会导致不必要的资源消耗。

Bridgic 提供了一个强大的 [`interact_with_human`](../../../../reference/bridgic-core/bridgic/core/automa/#bridgic.core.automa.Automa.interact_with_human) 机制来处理这种情况的中断恢复。这允许工作流或代理在发生此类事件时暂停并持久化其当前执行状态，等待反馈，然后恢复执行。

让我们实现一个由企业的 OA 系统自动触发的报销工作流，并在完成报销之前需要批准。可以在 GitHub 上下载 [源代码](https://github.com/bitsky-tech/bridgic-examples/blob/main/human_in_the_loop/reimbursement_automation.py)。

In [28]:
import os
import tempfile
from httpx import delete
from pydantic import BaseModel
from datetime import datetime
from bridgic.core.automa import GraphAutoma, worker, Snapshot
from bridgic.core.automa.args import From
from bridgic.core.automa.interaction import Event, InteractionFeedback, InteractionException

class ReimbursementRecord(BaseModel):
    request_id: int
    employee_id: int
    employee_name: str
    reimbursement_month: str
    reimbursement_amount: float
    description: str
    created_at: datetime
    updated_at: datetime

class AuditResult(BaseModel):
    request_id: int
    passed: bool
    audit_reason: str

class ReimbursementWorkflow(GraphAutoma):
    @worker(is_start=True)
    async def load_record(self, request_id: int):
        """
        The reimbursement workflow can be triggered by the OA system — for instance, when an employee submits a new reimbursement request. Each request is uniquely identified by a `request_id`, which is then used to retrieve the corresponding reimbursement record from the database. 
        """
        # Load the data from database.
        return await self.load_record_from_database(request_id)
    
    @worker(dependencies=["load_record"])
    async def audit_by_rules(self, record: ReimbursementRecord):
        """
        This method simulates the logic for automatically determining whether a reimbursement request complies with business rules.  

        Typical reasons for a reimbursement request failing the audit include:

        - Unusually large individual amounts
        - Excessive total amounts within a month
        - Expenses that do not meet reimbursement policies
        - Missing or invalid supporting documents
        - Duplicate submissions
        - Other non-compliant cases
        """
        if record.reimbursement_amount > 2500:
            return AuditResult(
                request_id=record.request_id,
                passed=False,
                audit_reason="The reimbursement amount {record.reimbursement_amount} exceeds the limit of 2500."
            )
        # TODO: Add more audit rules here.
        ...

        return AuditResult(
            request_id=record.request_id,
            passed=True,
            audit_reason="The reimbursement request passed the audit."
        )
    
    @worker(dependencies=["audit_by_rules"])
    async def execute_payment(self, result: AuditResult, record: ReimbursementRecord = From("load_record")):
        if not result.passed:
            print(f"The reimbursement request {record.request_id} failed the audit. Reason: {result.audit_reason}")
            return False
        
        # The reimbursement request {record.request_id} has passed the audit rules. Requesting approval from the manager...
        # human-in-the-loop: request approval from the manager.
        event = Event(
            event_type="request_approval",
            data={
                "reimbursement_record": record,
                "audit_result": result
            }        
        )
        feedback: InteractionFeedback = self.interact_with_human(event)
        if feedback.data == "yes":
            await self.lanuch_payment_transaction(record.request_id)
            print(f"The reimbursement request {record.request_id} has been approved. Payment transaction launched.")
            return True

        print(f"!!!The reimbursement request {record.request_id} has been rejected. Payment transaction not launched.\nRejection info:\n {feedback.data}")
        return False

    async def load_record_from_database(self, request_id: int):
        # Simulate a database query...
        return ReimbursementRecord(
            request_id=request_id,
            employee_id=888888,
            employee_name="John Doe",
            reimbursement_month="2025-10",
            reimbursement_amount=1024.00,
            description="Hotel expenses for a business trip",
            created_at=datetime(2025, 10, 11, 10, 0, 0),
            updated_at=datetime(2025, 10, 11, 10, 0, 0)
        )
    async def lanuch_payment_transaction(self, request_id: int):
        # Simulate a payment execution...
        ...

该工作流 `ReimbursementWorkflow` 包含三个步骤：

- `load_record`: 从数据库加载由 `request_id` 标识的报销记录。
- `audit_by_rules`: 根据预定义规则自动审核报销请求。
- `execute_payment`: 在报销请求通过审核规则后，请求经理（人类用户）批准。

在第三个步骤（`execute_payment`）中，调用 [`interact_with_human()`](../../../../reference/bridgic-core/bridgic/core/automa/#bridgic.core.automa.Automa.interact_with_human) 会发布一个事件并暂停工作流执行。

In [29]:
async def save_snapshot_to_database(snapshot: Snapshot):
    # Simulate a database storage using temporary files.
    temp_dir = tempfile.TemporaryDirectory()
    bytes_file = os.path.join(temp_dir.name, "reimbursement_workflow.bytes")
    version_file = os.path.join(temp_dir.name, "reimbursement_workflow.version")
    with open(bytes_file, "wb") as f:
        f.write(snapshot.serialized_bytes)
    with open(version_file, "w") as f:
        f.write(snapshot.serialization_version)

    return {
        "bytes_file": bytes_file,
        "version_file": version_file,
        "temp_dir": temp_dir,
    }


reimbursement_workflow = ReimbursementWorkflow()
try:
    await reimbursement_workflow.arun(request_id=123456)
except InteractionException as e:
    # The `ReimbursementWorkflow` instance has been paused and serialized to a snapshot.
    interaction_id = e.interactions[0].interaction_id
    record = e.interactions[0].event.data["reimbursement_record"]
    # Save the snapshot to the database.
    db_context = await save_snapshot_to_database(e.snapshot)
    print("The `ReimbursementWorkflow` instance has been paused and serialized to a snapshot.")
    print("The snapshot has been persisted to database.")

The `ReimbursementWorkflow` instance has been paused and serialized to a snapshot.
The snapshot has been persisted to database.


当调用 automa 实例的 [`arun`](../../../../reference/bridgic-core/bridgic/core/automa/#bridgic.core.automa.GraphAutoma.arun) 方法时，将会引发一个 [`InteractionException`](../../../../reference/bridgic-core/bridgic/core/automa/interaction/#bridgic.core.automa.interaction.InteractionException)，这是由于调用了 [`interact_with_human()`](../../../../reference/bridgic-core/bridgic/core/automa/#bridgic.core.automa.Automa.interact_with_human) 方法。

`InteractionException` 包含两个字段：

- `interactions`: 一个 [`Interaction`](../../../../reference/bridgic-core/bridgic/core/automa/interaction/#bridgic.core.automa.interaction.Interaction) 的列表，每个 `Interaction` 包含一个 `interaction_id` 和一个 `event`。
- `snapshot`: 一个 [`Snapshot`](../../../../reference/bridgic-core/bridgic/core/automa/#bridgic.core.automa.Snapshot) 实例，表示 Automa 当前状态的字节序列化。

然后，工作流 `ReimbursementWorkflow` 暂停，并且与该交互对应的快照被持久化到数据库中以便后续恢复。

In [ ]:
async def load_snapshot_from_database(db_context):
    # Simulate a database query using temporary files.
    bytes_file = db_context["bytes_file"]
    version_file = db_context["version_file"]
    temp_dir = db_context["temp_dir"]

    with open(bytes_file, "rb") as f:
        serialized_bytes = f.read()
    with open(version_file, "r") as f:
        serialization_version = f.read()
    snapshot = Snapshot(
        serialized_bytes=serialized_bytes, 
        serialization_version=serialization_version
    )
    return snapshot

print("Waiting for the manager's approval (It may take long time) ...")
human_feedback = input(
    "\n"
    "---------- Message to User ------------\n"
    "A reimbursement request has been submitted and audited by the system.\n"
    "Please check the details and give your approval or rejection.\n"

    "Reimbursement Request Details:\n"
    f"\n{record.model_dump_json(indent=4)}\n"
    "If you approve the request, please input 'yes'.\n"
    "Otherwise, please input 'no' or the reason for rejection.\n"
    "Your input: "
    )

# Load the snapshot from the database.
snapshot = await load_snapshot_from_database(db_context)
# Deserialize the `ReimbursementWorkflow` instance from the snapshot.
reimbursement_workflow = ReimbursementWorkflow.load_from_snapshot(snapshot)
print("-------------------------------------\n")
print("The `ReimbursementWorkflow` instance has been deserialized and loaded from the snapshot. It will resume to run immediately...")
feedback = InteractionFeedback(
    interaction_id=interaction_id,
    data=human_feedback
)
await reimbursement_workflow.arun(feedback_data=feedback)

Waiting for the manager's approval (It may take long time) ...
-------------------------------------

The `ReimbursementWorkflow` instance has been deserialized and loaded from the snapshot. It will resume to run immediately...
The reimbursement request 123456 has been approved. Payment transaction launched.


在经过一段时间后，用户可以完成报销工作流的审批交互。系统随后从数据库中检索序列化快照，并将其反序列化为 `ReimbursementWorkflow` 类的实例。接着，用户的决定——无论是批准还是拒绝——被封装到一个 [`InteractionFeedback`](../../../../reference/bridgic-core/bridgic/core/automa/interaction/#bridgic.core.automa.interaction.InteractionFeedback) 对象中，并再次调用 automa 的 arun 方法，以从之前暂停的状态恢复工作流执行。

当面临需要反馈但等待时间不确定的情况时，这种机制保存当前状态，并在未来合适的时刻重新进入。这不仅使系统能够释放本来会长时间占用的资源，还允许它在适当的时间被唤醒。

## 我们学到了什么？

Bridgic 提供了对任何形式的人在回路交互的灵活支持：

- [`request_feedback_async`](../../../../reference/bridgic-core/bridgic/core/automa/#bridgic.core.automa.Automa.request_feedback_async): 当事件必须在程序继续之前返回反馈时使用。程序在收到反馈之前保持阻塞状态。
- [`post_event`](../../../../reference/bridgic-core/bridgic/core/automa/#bridgic.core.automa.Automa.post_event): 当您只想通知或触发一个事件而不期望任何反馈时使用。主程序从不阻塞。
- [`interact_with_human`](../../../../reference/bridgic-core/bridgic/core/automa/#bridgic.core.automa.Automa.interact_with_human): 当需要反馈但可能会晚得多到达时使用。程序被挂起并持久化，并在反馈可用时立即恢复。